# Concurrency and Shared States

- Accessing **shared mutable variables** from **multi-threaded** dispatchers is discussed.
- Preventing concurrency issues is addressed using **volatile** variables, **atomic** types and **Mutex**.

In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import kotlin.coroutines.CoroutineContext
import java.util.concurrent.atomic.AtomicInteger
import kotlinx.coroutines.sync.Mutex
import kotlinx.coroutines.sync.withLock
import kotlin.system.measureTimeMillis

## Volatile Variables and `synchronized`

- **Use cases**: Simple signaling between threads, sharing state between threads, but with potential race conditions.
- A `volatile` variable is a variable that guarantees **visibility across threads**. This means that *any changes made to the variable by one thread will be immediately visible to other threads*.
- While `volatile` variables ensure visibility, they **DO NOT guarantee** any specific **ordering** of operations.
- It guarantees that any **read or write** to that variable will be directly **from and to main memory**, rather than using thread-local caches.
- Using `volatile` variables **can introduce some performance overhead**, as they require additional synchronization mechanisms.
- They **DO NOT guarantee** atomicity. For example, operations like incrementing a value `x++` are not atomic, even if `x` is declared `volatile`. We'll need to use **locks** or **atomic** variables (AtomicInteger, etc.) for that.
- A `synchronized` block can be used along with `volatile` variables that **blocks** other threads that attempt to access the same section of code while one thread is already executing it. This ensures atomicity.

In [2]:
// Examining volatile variables

@Volatile // In Kotlin `volatile` is an annotation.
var counter = 0 // This variable is shared.

fun main() = runBlocking {
    withContext(Dispatchers.Default) { // Using a multi-thread dispatcher
        val n = 1000
        repeat(n) { 
            launch { 
                counter++ // This action is not atomic. 
            }
        }
    }
    println("Counter = $counter [expected is 1000]") 
}

main()

Counter = 976 [expected is 1000]


In [3]:
// Using 'synchronized' with volatile variables

@Volatile 
var counter = 0 
// Let's create an object so 'synchronized' can apply a lock on it.
val lock = Any() 

fun main() = runBlocking {
    withContext(Dispatchers.Default) { // Using a multi-thread dispatcher
        val n = 1000
        repeat(n) { 
            launch { 
                /*
                The lock object is passed into the 'synchronized' block to ensure that only one thread
                at a time can execute the code block within the synchronized(lock) construct. 
                
                When one thread enters the synchronized block with this lock, 
                other threads trying to enter the same block are forced to 
                wait until the lock is released (i.e., when the first thread exits the block).

                It is common to use 'synchronized(this) {...}' when we want to apply
                a change to a volatile property of a class instance.
                */

                /*
                IMPORTANT: a synchronized block, blocks other threads that attempt to access
                the same section of code while one thread is already executing it.
                */
                synchronized(lock) {
                    counter++ // This action is not atomic. 
                }
            }
        }
    }
    println("Volatile Counter = $counter [expected is 1000]") 
}

main()

Volatile Counter = 1000 [expected is 1000]


## Atomic Variables

- **Use cases**: Implementing counters, flags, or other simple atomic operations.
- **Atomic** types in Kotlin provide a way to handle **multi-threaded operations** on **shared variables** safely *without needing complex synchronization mechanisms like locks*.
- Operations on atomic types are performed using **low-level CPU** instructions that ensure atomicity without requiring locks, thus **avoiding blocking other threads**.
- **Key Atomic Types**:
  1. `AtomicInteger`: For atomic operations on integers.
  2. `AtomicLong`: For atomic operations on long values.
  3. `AtomicBoolean`: For atomic operations on boolean values.
  4. `AtomicReference<T>`: For atomic operations on object references.
- **Common functions** for primitive atomic types:
  1. `incrementAndGet()`: Atomically increments the current value by 1 and returns the updated value.
  2. `decrementAndGet()`: Atomically decrements the current value by 1 and returns the updated value.
  3. `addAndGet(delta: Int)`: Atomically adds a specified value (delta) to the current value and returns the updated value.
  4. `getAndSet(newValue: Int)`: Atomically sets the new value and returns the previous value.
  5. `compareAndSet(expected: Int, newValue: Int)`: Atomically updates the value to the `newValue` if the current value equals the `expected` value.
- **Common functions** for primitive object references:
  1. `get()`: Returns the current value.
  2. `set(newValue: T)`: Atomically sets the new value.
  3. `compareAndSet(expected: T, newValue: T)`: Compares the current value with the `expected` value and sets a `newValue` if they match.
  4. `updateAndGet(updateFunction: (T) -> T)`: Atomically updates the current value using a given function and returns the updated value.

In [4]:
// Using 'AtomicInteger'

// Defining an atomic counter
var counter = AtomicInteger(0) 

fun main() = runBlocking {
    withContext(Dispatchers.Default) { // Using a multi-thread dispatcher
        val n = 1000
        repeat(n) { 
            launch { 
                counter.incrementAndGet() // This action is atomic. 
            }
        }
    }
    println("Atomic Counter = $counter [expected is 1000]") 
}

main()

Atomic Counter = 1000 [expected is 1000]


## Using `Mutex`

- **Use cases**: Protecting critical sections of code, preventing race conditions.
- **Preferred for Coroutines**: A **coroutine-friendly** lock that **suspends** the coroutine *instead of blocking the thread*, which is ideal for non-blocking synchronization. Use it with `withLock` to ensure safe access to shared resources in a coroutine context.
- `Mutex` is designed with coroutines in mind, providing a **safe** and idiomatic way to synchronize access to **shared mutable state** within the coroutine context. It fits well within the structured concurrency model of Kotlin, ensuring that locks are **automatically released** when the coroutine scope exits, **reducing the risk of deadlocks and resource leaks**.
- `Mutex` can be used with:
  1. `.withLock {...}`: This method is used to acquire the lock in a **suspending manner**. The block of code inside the curly braces will execute while holding the lock. Once the block completes, the lock is automatically released.
  2. `.tryLock()`:This method **attempts** to acquire the lock without blocking the coroutine. If the lock is already held by another coroutine, `tryLock` will return `false`.

In [5]:
// Using 'Mutex'

// Defining a mutex for synchronization
val mutex = Mutex()
var counter = 0

fun main() = runBlocking {
    withContext(Dispatchers.Default) { // Using a multi-thread dispatcher
        val n = 1000
        repeat(n) { 
            launch { 
                mutex.withLock { // Locking the mutex for safe access
                    counter++ // This action is now synchronized
                }
            }
        }
    }
    println("Mutex Counter = $counter [expected is 1000]") 
}

main()

Mutex Counter = 1000 [expected is 1000]


In [6]:
// Using 'tryLock'

val mutex = Mutex()
var counter = 0

fun main() = runBlocking {
    val n = 10_000
    withContext(Dispatchers.Default) {
        repeat(n) {
            launch {
                // Try to acquire the lock without blocking
                if (mutex.tryLock()) {
                    try {
                        counter++ // Safe access to sharedCounter
                    } finally {
                        mutex.unlock() // Ensure the lock is released
                    }
                } else {
                    // Handle the case where the lock could not be acquired
                    println("Could not acquire lock for counter increment.")
                }
            }
        }
    }
    println("Mutex (tryLock) Counter = $counter [expected is $n]")
}

main()

Could not acquire lock for counter increment.
Could not acquire lock for counter increment.
Could not acquire lock for counter increment.
Could not acquire lock for counter increment.
Could not acquire lock for counter increment.
Could not acquire lock for counter increment.
Could not acquire lock for counter increment.
Mutex (tryLock) Counter = 9993 [expected is 10000]


In [11]:
// Comparing performances

@Volatile 
var volCounter = 0 
val lock = Any() 

var atmCounter = AtomicInteger(0) 

val mutex = Mutex()
var mutCounter = 0

fun main() = runBlocking {
    // Volatile implementation
    val volTime = measureTimeMillis {
        withContext(Dispatchers.Default) { 
            val n = 1_000_000
            repeat(n) { 
                launch { synchronized(lock) { volCounter++ } }
            }
        }
    }
    println("Volatile time = $volTime millis")

    // Atomic implementation
    val atmTime = measureTimeMillis {
        withContext(Dispatchers.Default) { 
            val n = 1_000_000
            repeat(n) { 
                launch { atmCounter.incrementAndGet() }
            }
        }
    }
    println("Atomic time = $atmTime millis")

    // Mutex implementation
    val mtxTime = measureTimeMillis {
        withContext(Dispatchers.Default) { 
            val n = 1_000_000
            repeat(n) { 
                launch { mutex.withLock { mutCounter++ } }
            }
        }
    }
    println("Mutex time = $mtxTime millis") 

    /*
    IMPORTANT: While Mutex provides a more flexible and safe way to handle complex 
    synchronization scenarios (e.g., when multiple shared variables need to be updated atomically), 
    it may incur more overhead than simpler atomic operations or synchronized blocks, 
    particularly when contention is high.
    */
}

main()

Volatile time = 1007 millis
Atomic time = 650 millis
Mutex time = 1454 millis
